###import libraries and dependencies

In [1]:
from fisher_py.data.business import Scan
from fisher_py import RawFile
from sklearn.metrics.pairwise import cosine_similarity
import re
import numpy as np
import pandas as pd
import pickle 

###import raw file

In [3]:
raw = RawFile("C:/20231215_msz1718_MAbPac-06_Neo_ACN1_TreatmentA_biorep01_techrep01.raw")

###helper functions

In [4]:
def helper_regex(text):
    match = re.search(rf"{'Full'}\s+(\w+)", text)
    if match:
        return match.group(1)
    return None

### cast function for 0.1 amu resolution

In [6]:
def cast_func(scan_number):
    scan_masses = []
    scan_intensities = []
    data_masses = []
    data_intensities = [] 


    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=scan_number)
    precursor = float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1])
    scan_masses = raw_scan.preferred_masses
    scan_intensities = raw_scan.preferred_intensities

    for i in range(4000, 20000): 
        data_intensities.append(0) 
        data_masses.append(i/10)

    for j in range(0,len(scan_masses)):
        for i in range(0,len(data_masses)):
            if (i+ 4000 + 1)/10 > scan_masses[j] > (i + 4000)/10: 
                data_intensities[i] = data_intensities[i] + scan_intensities[j]

    # for i in range(0,16000):
    #     if (precursor -10) < data_masses[i] < (precursor +10): data_intensities[i] = 0
        
    return(list(data_intensities))

###cast function fro 1 amu resolution

In [45]:
def cast_func_1amu(scan_number):
    scan_masses = []
    scan_intensities = []
    data_masses = []
    data_intensities = [] 
    scan_masses1 = []
    scan_intensities1 = []


    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=scan_number)
    precursor = float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1])
  
    scan_masses = raw_scan.preferred_masses
    scan_intensities = raw_scan.preferred_intensities
   
    for k in range(0, len(scan_intensities)):
        if scan_intensities[k] > (5/100)*max(scan_intensities): 
            scan_intensities1.append(scan_intensities[k])
            scan_masses1.append(scan_masses[k])


    for i in range(400, 2000): 
        data_intensities.append(0) 
        data_masses.append(i)

    for j in range(0,len(scan_masses1)):
        for i in range(0,len(data_masses)):
            if (i+ 400 + 1) > scan_masses1[j] > (i + 400): 
                data_intensities[i] = data_intensities[i] + scan_intensities1[j]

    # for i in range(0,16000):
    #     if (precursor -10) < data_masses[i] < (precursor +10): data_intensities[i] = 0
        
    return(list(data_intensities))

### generating distances against a single scan

In [ ]:
ref_scan = cast_func_1amu(10244)
scan_id = []
scan_sim = []
precursor = []

for i in range(1, raw.number_of_scans):
    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
    if str(helper_regex(raw_scan.scan_type)) == 'ms2':
        X = [ref_scan, cast_func_1amu(i)]
        result = round(cosine_similarity(X)[1][0], 2)       
        precursor.append(float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1]))
        scan_id.append(i)
        scan_sim.append(result)
        print(float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1]), "in scan", i, "is", result)

np.savetxt("outputall.csv", np.transpose(np.vstack((np.array(scan_id), np.array(precursor), np.array(scan_sim)))) , delimiter=",")

### generate a database

In [ ]:
database = []

for i in range(1, raw.number_of_scans):
    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
    if str(helper_regex(raw_scan.scan_type)) == 'ms2':
        print(i)
        database.append(cast_func_1amu(i))



In [9]:
with open('database', 'wb') as f: 
     pickle.dump(database, f) 

###search against a database

In [ ]:
with open('database', 'rb') as f: 
     database = pickle.load(f) 

for i in range(0, len(database)):
    X = [database[0], database[i]]
    result = round(cosine_similarity(X)[1][0], 2)  
    print(result)

In [25]:
raw_scan = Scan.from_file(raw._raw_file_access, scan_number=1000)

max(raw_scan.preferred_masses)



1907.007568359375